<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Inicialização de Bibliotecas, Pastas e Foco da Camera</p>

In [1]:
import cv2
import numpy as np
import os
import time
import functions as f
from collections import deque

os.makedirs("../data", exist_ok=True)
os.makedirs("../data/inputs", exist_ok=True)

for i in range(6):
    os.makedirs(f"../data/inputs/{i}", exist_ok=True)

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0.25)
cap.set(cv2.CAP_PROP_EXPOSURE, -1)

cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)
cap.set(cv2.CAP_PROP_FOCUS, 10)

print("Exposição:", cap.get(cv2.CAP_PROP_EXPOSURE))
print("Foco:", cap.get(cv2.CAP_PROP_FOCUS))

Exposição: -6.0
Foco: -1.0


<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Bloco Principal</p>

In [2]:
cap = cv2.VideoCapture(0)
bbox = False
grid = True
division = 16

while True:
    ret,frame_original = cap.read()
    frame_original = f.mirroring(frame_original)
    frame = frame_original.copy()
    frame = f.gray_scale(frame)
    frame = f.background_subtraction(cam_frame=frame,start_time=3,limiar=15)
    frame,vector = f.down_sampling(frame,division=division)
    frame = f.binarization(frame,limiar=5)
    vector = f.binarization(vector,limiar=5)
    original_view = frame_original.copy()
    _,vector_positioning = f.bounding_box(frame,original_view,division=division)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    elif cv2.waitKey(1) & 0xFF == ord('1'):
        f.background_subtraction.background = f.gray_scale(frame_original.copy())
        frame_original = np.full_like(frame_original,255)


    if not ret:
        break

    vector = vector[vector_positioning[0]:vector_positioning[1], vector_positioning[2]:vector_positioning[3]]


    cv2.imshow('Original',original_view)
    cv2.imshow('Visualizer',frame)


cap.release()
cv2.destroyAllWindows()

<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Bloco de Trabalho</p>

In [3]:
cap = cv2.VideoCapture(0)
bbox = False
grid = True
division = 16

while True:
    ret,frame_original = cap.read()
    frame_original = f.mirroring(frame_original)
    frame = frame_original.copy()
    frame = f.gray_scale(frame)
    frame = f.background_subtraction(cam_frame=frame,start_time=3,limiar=15)
    frame,vector = f.down_sampling(frame,division=division)
    frame = f.binarization(frame,limiar=5)
    vector = f.binarization(vector,limiar=5)
    original_view = frame_original.copy()
    original_view,vector_positioning = f.bounding_box(frame,original_view,division=division)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    elif cv2.waitKey(1) & 0xFF == ord('1'):
        f.background_subtraction.background = f.gray_scale(frame_original.copy())
        frame_original = np.full_like(frame_original,255)


    if not ret:
        break

    vector = vector[vector_positioning[0]:vector_positioning[1], vector_positioning[2]:vector_positioning[3]]


    cv2.imshow('Original',original_view)
    cv2.imshow('Visualizer',frame)


cap.release()
cv2.destroyAllWindows()

<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Bloco Teste wave.gif
</p>

In [4]:
while True:
    cam = cv2.VideoCapture(r'../readme_media/wave.gif')
    while True:
        ret,frame = cam.read()
        
        if cv2.waitKey(66) & 0xFF == ord('q'):
            break
        if not ret:
            break
        frame_vizualizer = f.grid(frame)
        cv2.imshow('Vid',frame_vizualizer)
    if cv2.waitKey(500) & 0xFF == ord('q'):
        break
    
cam.release()
cv2.destroyAllWindows()